# Part I: Basics
As an opening, we first mention some important concepts in deep learning, including
- Artificial Neural Network
- Automatic Differentiation and Backpropogation
- Monte Carlo Estimation 
- Stochastic Gradient Descent.

Given the extensive nature of each subsection, a comprehensive coverage is beyond the scope. Instead, we will pick certain subjects, explain some important ideas and theorems that suuport these mechanisms, and provide a simple example as demonstration.

## Artificial Neural Network

Artificial Neural Network, or abreviated neural network in machine learning context, is the core of deep learning. The name and structure are inspired by human brain, mimicking the interactions of biological neurons. 

A neural network consists of an input layer, one or multiple hidden layers, and an output layer. As a toy example, a neural network is like:

In [ ]:
# Don't execute
nn.Linear(input_dim,output_dim) 

which performs the linear transformations on $X_{n\times i}\in\mathbb{R}^{n\times i}$:
$$
Y_{n\times o} = X_{n\times i}W_{i\times o}+b_{o}
$$
where $W_{i\times o}$ is a weight matrix and $b_o$ is a bias vector. 

Similarly, we can implement a neural network with three layers as

In [ ]:
# Don't execute
nn.Sequential(
    nn.Linear(input_dim,hidden_1),
    nn.Linear(hidden_1,hidden_2),
    nn.Linear(hidden_2,output_dim)
)

In practice, one can implement the neural network with different tricks: adjusting the depth (number of layers) of neural network or width (number of neurons) of a layer, using dropout [number], using batch normalization [number], or even choosing another activation function if needed.

The universal approximation theorem is a key factor contributing to the widespread application of neural networks across various fields and scenarios. The universal approximation theorem originally is given by Cybenko in 1989 [number] using sigmoidal functions
$$
\begin{equation}
\sigma(x)=
\begin{cases}
1, & x\to \infty \\
0, & x\to-\infty
\end{cases}.
\end{equation}
$$
Denote $I_n=[0,1]^n$ the $n$-dimensional unit cube and $C(I_n,\mathbb{R})$ space of continuous functions from $I_n$ to $\mathbb{R}$.

**Theorem (Universal Approximation Theorem, G.Cybenko).** Let $\sigma$ be any continuous sigmoidal function, $f\in C(I_n,\mathbb{R})$. Then finite sums of the form
$$
G^N(x)=\sum_{j=1}^N \alpha_j\sigma(w_j^T x+\theta_j)
$$
are dense in the space of continuous functions $C(I_n,\mathbb{R})$. In other words, given any $f\in C(I_n,\mathbb{R})$, there exists $N\in\mathbb{N}$ such that
$$
|G^N(x)-f(x)|<\epsilon \quad \text{for all } x\in I_n.
$$
This original universal approximation theorem says a neural network with one layer and arbitrary width can approximate any continuous function on $[0,1]^n$, where $N$ is the number of neurons. Now there are many variants and generalizations of the theorem, for example [number],[number]. Another insight of the universality of neural network can be interpreted by ordinary differential equations, as referenced in [number].

## Automatic Differentiation and Backpropogation
Automatic differentiation is widely used for deep learning optimization. In a nutshell, automatic differentiation is a clever way of performing chain rule without manually computing derivatives manually. 

Let $f=f_u\circ f_{u-1}\circ \cdots \circ f_1$ be our neural network. Each $f_i$ is differentiable. Chain rule of differentiation tells us how to compute the derivative:
$$
\frac{df}{dx} = \frac{df_u}{df_{u-1}}\cdots\frac{df_{2}}{df_{1}}\frac{df_1}{dx}.
$$

## Monte Carlo Estimation

Let $p(\mathbf{x},\boldsymbol{\theta})$ be some probability distribution depending on a collection $\boldsymbol{\theta}$ of parameters. Consider the expectation of the form 
$$
\mathcal{F}(\boldsymbol{\theta})=\int p(\mathbf{x},\boldsymbol{\theta})f(\mathbf{x},\boldsymbol{\phi})d\mathbf{x}=\mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[f(\mathbf{x},\boldsymbol{\phi})\right]
$$
where $\mathbf{x}$ is the input of objective $f$ with probability $p(\mathbf{x},\boldsymbol{\theta})$, and $\boldsymbol{\phi}$ is a set of the parameters of $f$. Of course, $\boldsymbol{\phi}$ might be equal to $\boldsymbol{\theta}$. We are interested in learning the parameters $\boldsymbol{\theta}$, which requires the computation of the gradient of $\mathcal{F}(\boldsymbol{\theta})$ with respect to $\theta$:
$$
\nabla_{\boldsymbol{\theta}}\mathcal{F}(\boldsymbol{\theta})=\nabla_{\boldsymbol{\theta}}\mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[f(\mathbf{x},\boldsymbol{\phi})\right].
$$
The expectation in general is intractable because the distribution $p(\mathbf{x},\boldsymbol{\theta})$ might be high-dimensional, in deep learning, easily in the order of thousands or even more of dimensions, and very complicated. Moreover, the function $f$ might be non-differentiable, or a black-box function which the output is all we observe, artificial neural network for example. 


The Monte Carlo Method provides another insight of this sort of impossible calculation. Instead of computing the closed form of the integral directly, we draw i.i.d. samples $\hat{\mathbf{x}}^{(1)},...,\hat{\mathbf{x}}^{(S)}$ from $p(\mathbf{x},\boldsymbol{\theta})$, and approximate the integral with the average of $f(\hat{\mathbf{x}}^{(i)},\boldsymbol{\phi})$, called a Monte Carlo estimator:
$$
\bar{\mathcal{F}}_S=\frac{1}{S}\sum_{i=1}^S f(\hat{\mathbf{x}}^{(i)},\boldsymbol{\phi}).
$$
Although $\bar{\mathcal{F}}_S$ is still a random variable because it depends on random variables $\hat{\mathbf{x}}^{(1)},...,\hat{\mathbf{x}}^{(S)}$, now it is equiped with desirable properties:

**Unbiasedness.**
$$
\begin{aligned}
\mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[\bar{\mathcal{F}}_S\right] & = \mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[\frac{1}{S}\sum_{i=1}^S f(\hat{\mathbf{x}}^{(i)},\boldsymbol{\phi})\right] = \frac{1}{S}\sum_{i=1}^S\mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[ f(\hat{\mathbf{x}}^{(i)},\boldsymbol{\phi})\right] \\
& = \frac{1}{S}\sum_{i=1}^S\mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[f(\mathbf{x},\boldsymbol{\phi})\right] = \mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[f(\mathbf{x},\boldsymbol{\phi})\right].
\end{aligned}
$$
Unbiasedness is always preferred because it allows us to guarantee the convergence of a stochastic optimisation procedure.

**Consistency.** 
By strong law of large numbers, the random variable $\bar{\mathcal{F}}_S$ converges to $\mathbb{E}_{\mathbf{x}\sim p(\mathbf{x},\boldsymbol{\theta})}\left[f(\mathbf{x},\boldsymbol{\phi})\right]$ almost surely as the number of samples $S$ increases.

Monte Carlo estimation provides a convenient approach to approximate expectations. For example, in generative adversarial networks, the discriminator is updated with the loss function
$$
-\mathbb{E}_{\hat{\mathbf{x}}\sim p_{data}}\left[\log D(\hat{\mathbf{x}})\right]-\mathbb{E}_{\hat{\mathbf{z}}\sim \mathcal{N}(0,\mathbf{I})} \left[\log (1-D(G(\hat{\mathbf{z}})))\right].
$$
where $D$ is the discriminator network, $G$ is the generator network, $x$ is a datapoint sampled from data distribution $p_{data} $ and $z$ is a random noise vector sampled from $\mathcal{N}(0,\mathbf{I})$. With Monte Carlo estimation, instead of calculating two intractable integrals, we only need to calculate the average
$$
-\frac{1}{m}\sum_{i=1}^m \ln D(\hat{\mathbf{x}}^{(i)})-\frac{1}{m}\sum_{i=1}^m \ln(1-D(G(\hat{\mathbf{z}}^{(i)}))).
$$
The quantity $m$, or batch size, is a hyperparameter. Even though larger $m$ gives better approximation of the expectation, too large $m$ may induce inferior performance in the optimization procedure [number]. The only way to find "better" $m$ is through numerous trials with patience. 

## (Minibatch) Stochastic Gradient Descent
Nowadays, people usually prefer minibatch stochastic gradient descent to optimize neural networks, as it is more computationally efficient than gradient descent or stochastic gradient descent. 

Let $\ell$ be our loss function. Let $n$ be the number of training data point, $b$ the batch size, $\boldsymbol{\theta^0}\in\mathbb{R}^d$ the initial parameter of our neural network and $(\alpha_t)_{t\in\mathbb{N}}$ a sequence of step size, or learning rate. Given a subsest $B\subset \{1,...,n\}$, we define
$$
\nabla_{\boldsymbol{\theta^t}} \ell_B(\boldsymbol{\theta^t})=\frac{1}{|B|}\sum_{i\in B} \nabla_{\boldsymbol{\theta^t}} \ell_i(\boldsymbol{\theta^t})
$$
The minibatch stochastic gradient descent algorithm is given by 
$$
\begin{aligned}
& B_t\subset\{1,...,n\} \quad\quad \text{Sampled uniformly among sets of size $b$} \\
& \boldsymbol{\theta^{t+1}} = \boldsymbol{\theta^t}-\alpha_t \nabla_{\boldsymbol{\theta^t}} \ell_B(\boldsymbol{\theta^t}).
\end{aligned}
$$
Note that $\nabla_{\boldsymbol{\theta^t}} \ell_B(\boldsymbol{\theta^t})$ is an unbiased estimator
$$
\mathbb{E}_{b}\left[\nabla_{\boldsymbol{\theta^t}} \ell_B(\boldsymbol{\theta^t})\right] = \frac{1}{{n \choose b}} \sum_{\substack{B\subset\{1,...,n\} \\ |B|=b}} \nabla_{\boldsymbol{\theta^t}} \ell_B(\boldsymbol{\theta^t}) = \nabla_{\boldsymbol{\theta^t}} \ell(\boldsymbol{\theta^t})
$$
because each batch is sampled with probability $\frac{1}{{n \choose b}}$. For details on the convergence theorem applicable to convex and smooth functions, please refer to [number].


# References
- N. Srivastava et al. Salakhutdinov. Dropout: A Simple Way to Prevent Neural Networks from Overfitting. In *Journal of Machine Learning Research*, 2014.
- S. Ioffe et al. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift. In *Journal of Machine Learning Research*, 2015.
- D. Masters et al. Revisiting Small Batch Training for Deep Neural Networks, *arXiv:1804.07612*, 2018.
- G. Cybenko. Approximation by superpositions of a sigmoidal function. In *Springer*, 1989.
- A. Pinkus. Approximation theory of the MLP model in neural networks, 1999.
- DX Zhou. Universality of deep convolutional neural networks, 2020.
- P. Kidger. On Neural Differential Equations, Oxford University, 2022.
- S. Mohamed et al. Monte Carlo Gradient Estimation in Machine Learning. In *Journal of Machine Learning Research*, 2020.
- S. Ruder. An overview of gradient descent optimization algorithms, 2016.
- S. Bubeck. Convex Optimization: Algorithms and Complexity, *Foundations and Trends® in Machine Learning*, 2014.